In [10]:
import os
import re
# os.environ["OPENAI_API_KEY"] = "sk-your-api-key-here"

In [14]:
from importlib import reload

import httpx
from openai import OpenAI

import src.agents as agents

In [6]:
client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

## ReAct prompt

In [11]:
react_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Django
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris
""".strip()

actions_re = re.compile('^Action: (\w+): (.*)')

## Actions

In [21]:
def wikipedia(q):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

def calculate(query: str):
    return eval(query)

known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate
}

In [8]:
reload(agents)

my_agent = agents.BaseAgent(client=client)
my_agent("hello")

'Hello! How can I assist you today?'

In [27]:
def query_agent(query: str, max_turns: int = 5):
    i = 0
    bot = agents.BaseAgent(client=client, system_prompt=react_prompt)
    next_prompt = query

    while i < max_turns:
        i += 1
        res = bot(next_prompt)
        print(res)
        
        actions = []
        for action in res.split('\n'):
            if action in known_actions:
                actions.append(actions_re.match(action))

        if actions:
            actions, action_input = actions[0].groups()
            if actions not in known_actions:
                raise ValueError(f"Unknown action: {actions}")
            print(f"Running action: {actions} with input: {action_input}")
            observations = known_actions[actions](action_input)
            print(f"Observation: {observations}")
            next_prompt = f"Observation: {observations}"
        else:
            return res

print(query_agent("What does England share borders with?"))

Thought: England is part of the United Kingdom, so it shares borders with other countries within the UK. I should look up what borders England has, specifically with Scotland and Wales. Additionally, it may be useful to confirm the geographical context from Wikipedia.  
Action: wikipedia: England borders  
PAUSE
Thought: England is part of the United Kingdom, so it shares borders with other countries within the UK. I should look up what borders England has, specifically with Scotland and Wales. Additionally, it may be useful to confirm the geographical context from Wikipedia.  
Action: wikipedia: England borders  
PAUSE
